##  <font color = "navy">WEATHER FORECAST IN AUSTRALIA: Modélisation deep learning</font>
Notebook <b>3/3</b>
***

<font color = "purple"><b>Objectif: Réaliser un modèle de deep learning capable de détecter les jours de pluie à J+1 avec une précision satisfaisante.</b></font>

<b> Résumé du notebook : </b>
- Data cleaning, preprocessing des données (notamment encodage, et rééquilibrage) ;
- Modélisation deep learning ;
- Essai d'amélioration des performances du modèle.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import warnings
from sklearn.preprocessing import LabelEncoder
from IPython.display import Markdown, display
warnings.filterwarnings('ignore')
sns.set_theme({'legend.frameon':True})
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
import itertools 
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

#pip install --upgrade tensorflow
#pip install pydot
#pip install graphviz

In [2]:
#Import d'une version prête à l'emploi (preprocessing: Cf notebook 2/4)
data = pd.read_csv("weatherAUS_DL.csv")

#Création d'un df de secours pour avoir une trace des données brutes.

df_saved = pd.read_csv("weatherAUS_DL.csv")

pd.set_option("display.max_columns", None)

feats = data.drop(["RainTomorrow", "Date", "Year", "Month", "Day", "State", "Location"], axis = 1)
target = data["RainTomorrow"]
encoder = LabelEncoder()

Y = encoder.fit_transform(target)

X_train, X_test, y_train, y_test = train_test_split(feats, Y, test_size = 0.2, random_state = 55) 

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
display(Markdown("""<h2><center><u>Réalisation d'un premier modèle de deep learning</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))
EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)
print(score)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Réel"], colnames = ["Prédiction"]))

<h2><center><u>Réalisation d'un premier modèle de deep learning</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 926us/step - loss: 0.3535 - accuracy: 0.8469 - val_loss: 0.3416 - val_accuracy: 0.8529
Epoch 2/6
3229/3229 [==============================] - 3s 877us/step - loss: 0.3447 - accuracy: 0.8509 - val_loss: 0.3368 - val_accuracy: 0.8539
Epoch 3/6
3229/3229 [==============================] - 3s 873us/step - loss: 0.3420 - accuracy: 0.8516 - val_loss: 0.3344 - val_accuracy: 0.8550
Epoch 4/6
3229/3229 [==============================] - 3s 888us/step - loss: 0.3402 - accuracy: 0.8531 - val_loss: 0.3333 - val_accuracy: 0.8563
Epoch 5/6
3229/3229 [==============================] - 3s 881us/step - loss: 0.3388 - accuracy: 0.8531 - val_loss: 0.3328 - val_accuracy: 0.8569
Epoch 6/6
808/808 [==============================] - 0s 599us/step - loss: 0.3356 - accuracy: 0.8544
[0.3356141746044159, 0.8544102907180786]


<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>70.46 %</b> (<i>recall = <b>56.08 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.55 %</b> (<i>recall = <b>93.53 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.44 %</b>.</li></ul>
Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.89      0.94      0.91     20250
           1       0.70      0.56      0.62      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.75      0.77     25826
weighted avg       0.85      0.85      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Prédiction      0     1
Réel                   
0           18939  1311
1            2449  3127


In [4]:
display(Markdown("""<h2><center><u>Effet du nombre de neurones</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 250
UNITS2 = 500
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")


x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Réel"], colnames = ["Prédiction"]))

<h2><center><u>Effet du nombre de neurones</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3498 - accuracy: 0.8488 - val_loss: 0.3374 - val_accuracy: 0.8556
Epoch 2/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3406 - accuracy: 0.8523 - val_loss: 0.3337 - val_accuracy: 0.8565
Epoch 3/6
3229/3229 [==============================] - 5s 1ms/step - loss: 0.3370 - accuracy: 0.8534 - val_loss: 0.3323 - val_accuracy: 0.8565
Epoch 4/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3342 - accuracy: 0.8547 - val_loss: 0.3345 - val_accuracy: 0.8560
Epoch 5/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3315 - accuracy: 0.8566 - val_loss: 0.3311 - val_accuracy: 0.8585
Epoch 6/6
808/808 [==============================] - 1s 774us/step - loss: 0.3303 - accuracy: 0.8594


<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>250 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>500 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 250)               5750      
                                                                 
 Dense_2 (Dense)             (None, 500)               125500    
                                                                 
 Dense_3 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 132,252
Trainable params: 132,252
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>73.14 %</b> (<i>recall = <b>55.09 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.42 %</b> (<i>recall = <b>94.43 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.94 %</b>.</li></ul>
Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.73      0.55      0.63      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.75      0.77     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Prédiction      0     1
Réel                   
0           19122  1128
1            2504  3072


In [5]:
display(Markdown("""<h2><center><u>Effet du nombre d'épochs</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 18
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Réel"], colnames = ["Prédiction"]))

<h2><center><u>Effet du nombre d'épochs</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/18
3229/3229 [==============================] - 3s 909us/step - loss: 0.3530 - accuracy: 0.8479 - val_loss: 0.3395 - val_accuracy: 0.8540
Epoch 2/18
3229/3229 [==============================] - 3s 934us/step - loss: 0.3440 - accuracy: 0.8512 - val_loss: 0.3374 - val_accuracy: 0.8547
Epoch 3/18
3229/3229 [==============================] - 3s 951us/step - loss: 0.3416 - accuracy: 0.8520 - val_loss: 0.3451 - val_accuracy: 0.8517
Epoch 4/18
3229/3229 [==============================] - 3s 867us/step - loss: 0.3397 - accuracy: 0.8523 - val_loss: 0.3357 - val_accuracy: 0.8558
Epoch 5/18
3229/3229 [==============================] - 3s 872us/step - loss: 0.3382 - accuracy: 0.8525 - val_loss: 0.3339 - val_accuracy: 0.8558
Epoch 6/18
3229/3229 [==============================] - 3s 899us/step - loss: 0.3368 - accuracy: 0.8533 - val_loss: 0.3331 - val_accuracy: 0.8556
Epoch 7/18
3229/3229 [==============================] - 3s 867us/step - loss: 0.3357 - accuracy: 0.8540 - val_loss: 0.3335 -

<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>18 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>73.47 %</b> (<i>recall = <b>53.14 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.01 %</b> (<i>recall = <b>94.72 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.74 %</b>.</li></ul>
L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     20250
           1       0.73      0.53      0.62      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.74      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Prédiction      0     1
Réel                   
0           19180  1070
1            2613  2963


In [6]:
display(Markdown("""<h2><center><u>Effet de la taille des batchs</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 320

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()
                 
display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle initial) 
ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Réel"], colnames = ["Prédiction"]))

<h2><center><u>Effet de la taille des batchs</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
323/323 [==============================] - 1s 2ms/step - loss: 0.3824 - accuracy: 0.8358 - val_loss: 0.3416 - val_accuracy: 0.8525
Epoch 2/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3461 - accuracy: 0.8505 - val_loss: 0.3388 - val_accuracy: 0.8542
Epoch 3/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3437 - accuracy: 0.8513 - val_loss: 0.3378 - val_accuracy: 0.8544
Epoch 4/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3423 - accuracy: 0.8521 - val_loss: 0.3360 - val_accuracy: 0.8550
Epoch 5/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3408 - accuracy: 0.8521 - val_loss: 0.3364 - val_accuracy: 0.8543
Epoch 6/6
808/808 [==============================] - 0s 595us/step - loss: 0.3340 - accuracy: 0.8552


<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>320</b>.</li></ul>


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>72.98 %</b> (<i>recall = <b>52.28 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.81 %</b> (<i>recall = <b>94.67 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.52 %</b>.</li></ul>
L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle initial) 
ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     20250
           1       0.73      0.52      0.61      5576

    accuracy                           0.86     25826
   macro avg       0.80      0.73      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Prédiction      0     1
Réel                   
0           19171  1079
1            2661  2915


In [7]:
display(Markdown("""<h2><center><u>Effet des fonctions d'activation</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "tanh"
ACTIV2 = "tanh"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Réel"], colnames = ["Prédiction"]))

<h2><center><u>Effet des fonctions d'activation</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 931us/step - loss: 0.3597 - accuracy: 0.8441 - val_loss: 0.3471 - val_accuracy: 0.8519
Epoch 2/6
3229/3229 [==============================] - 3s 870us/step - loss: 0.3542 - accuracy: 0.8463 - val_loss: 0.3475 - val_accuracy: 0.8495
Epoch 3/6
3229/3229 [==============================] - 3s 873us/step - loss: 0.3513 - accuracy: 0.8473 - val_loss: 0.3428 - val_accuracy: 0.8532
Epoch 4/6
3229/3229 [==============================] - 3s 886us/step - loss: 0.3480 - accuracy: 0.8487 - val_loss: 0.3411 - val_accuracy: 0.8543
Epoch 5/6
3229/3229 [==============================] - 3s 892us/step - loss: 0.3458 - accuracy: 0.8496 - val_loss: 0.3415 - val_accuracy: 0.8528
Epoch 6/6
808/808 [==============================] - 0s 589us/step - loss: 0.3390 - accuracy: 0.8534


<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>70.23 %</b> (<i>recall = <b>55.72 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.46 %</b> (<i>recall = <b>93.5 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.34 %</b>.</li></ul>
L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.93      0.91     20250
           1       0.70      0.56      0.62      5576

    accuracy                           0.85     25826
   macro avg       0.79      0.75      0.77     25826
weighted avg       0.85      0.85      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Prédiction      0     1
Réel                   
0           18933  1317
1            2469  3107


In [8]:
display(Markdown("""<h2><center><u>Effet du nombre de couches de neurones</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = 50
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = "relu"
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITS3, activation = ACTIV3, kernel_initializer = "normal", name = "Dense_3")
dense4 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_4")

x = dense1(inputs)
x = dense2(x)
x = dense3(x)
outputs = dense4(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la troisième avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, UNITS3, ACTIV3, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre de couches de neurones</u></h2></center>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 955us/step - loss: 0.3555 - accuracy: 0.8460 - val_loss: 0.3388 - val_accuracy: 0.8543
Epoch 2/6
3229/3229 [==============================] - 3s 953us/step - loss: 0.3445 - accuracy: 0.8506 - val_loss: 0.3374 - val_accuracy: 0.8544
Epoch 3/6
3229/3229 [==============================] - 3s 932us/step - loss: 0.3428 - accuracy: 0.8516 - val_loss: 0.3362 - val_accuracy: 0.8541
Epoch 4/6
3229/3229 [==============================] - 3s 928us/step - loss: 0.3414 - accuracy: 0.8526 - val_loss: 0.3338 - val_accuracy: 0.8556
Epoch 5/6
3229/3229 [==============================] - 3s 927us/step - loss: 0.3395 - accuracy: 0.8530 - val_loss: 0.3352 - val_accuracy: 0.8550
Epoch 6/6
808/808 [==============================] - 1s 605us/step - loss: 0.3332 - accuracy: 0.8559


<h4><u>Récapitulatif du modèle :</h4></u>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la troisième avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 50)                2550      
                                                                 
 Dense_4 (Dense)             (None, 2)                 102       
                                                                 
Total params: 4,527
Trainable params: 4,527
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</h4></u>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>73.12 %</b> (<i>recall = <b>52.6 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.88 %</b> (<i>recall = <b>94.68 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.59 %</b>.</li></ul>
L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     20250
           1       0.73      0.53      0.61      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.74      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19172  1078
1                2643  2933
